# Test Avatar Input Generation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model, sample_traces
import numpy as np
from tqdm import tqdm
import random

# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)

In [4]:
ocel.log.log

,event_id,event_activity,event_timestamp,event_start_timestamp,order,item,delivery
event_index,,,,,,,
0,0,Place Order,2022-10-20 16:56:19.864004,2022-10-20 16:56:19.864004,[order1],"[item2, item3]",[]
1,1,Confirm Order,2022-10-20 16:56:20.864004,2022-10-20 16:56:20.864004,[order1],"[item2, item3]",[]
2,2,Pick Item,2022-10-20 17:01:51.864004,2022-10-20 17:01:51.864004,[],[item3],[]
3,3,Pick Item,2022-10-20 17:01:51.864004,2022-10-20 17:01:51.864004,[],[item2],[]
4,4,Pay Order,2022-10-20 17:01:51.864004,2022-10-20 17:01:51.864004,[order1],[],[]
...,...,...,...,...,...,...,...
2386,2386,Start Route,2022-12-17 23:00:15.871959,2022-12-17 23:00:15.871959,[],"[item199, item196]",[delivery64]
2387,2387,End Route,2022-12-18 04:01:04.871959,2022-12-18 04:01:04.871959,[],"[item199, item196]",[delivery64]
2388,2388,Fuel Car,2022-12-18 06:59:45.871959,2022-12-18 06:59:45.871959,[],[],[delivery193]


In [5]:
# since the process execution mappings have lists of length one,
# we create another dictionary that only contains the the value inside the list to be able to derive the case
mapping_dict = {key: ocel.process_execution_mappings[key][0] for key in ocel.process_execution_mappings}
# we generate a new column in the class (log) that contains the process execution (case) number via the generated dictionary
ocel.log.log['event_execution'] = ocel.log.log.index.map(mapping_dict)

In [6]:
ocel.log.log

,event_id,event_activity,event_timestamp,event_start_timestamp,order,item,delivery,event_execution
event_index,,,,,,,,
0,0,Place Order,2022-10-20 16:56:19.864004,2022-10-20 16:56:19.864004,[order1],"[item2, item3]",[],33
1,1,Confirm Order,2022-10-20 16:56:20.864004,2022-10-20 16:56:20.864004,[order1],"[item2, item3]",[],33
2,2,Pick Item,2022-10-20 17:01:51.864004,2022-10-20 17:01:51.864004,[],[item3],[],33
3,3,Pick Item,2022-10-20 17:01:51.864004,2022-10-20 17:01:51.864004,[],[item2],[],33
4,4,Pay Order,2022-10-20 17:01:51.864004,2022-10-20 17:01:51.864004,[order1],[],[],33
...,...,...,...,...,...,...,...,...
2386,2386,Start Route,2022-12-17 23:00:15.871959,2022-12-17 23:00:15.871959,[],"[item199, item196]",[delivery64],0
2387,2387,End Route,2022-12-18 04:01:04.871959,2022-12-18 04:01:04.871959,[],"[item199, item196]",[delivery64],0
2388,2388,Fuel Car,2022-12-18 06:59:45.871959,2022-12-18 06:59:45.871959,[],[],[delivery193],0


In [8]:
# generate a grouped df such that we can iterate through the log case by case (sort by timestamp to ensure the correct process sequence)
grouped_df = ocel.log.log.sort_values('event_timestamp').groupby('event_execution')

In [9]:
sorted_log = ocel.log.log.sort_values(['event_execution','event_timestamp'])

In [10]:
sorted_log

,event_id,event_activity,event_timestamp,event_start_timestamp,order,item,delivery,event_execution
event_index,,,,,,,,
5,5,Place Order,2022-10-20 17:06:18.864004,2022-10-20 17:06:18.864004,[order2],"[item4, item5]",[],0
6,6,Confirm Order,2022-10-20 17:06:19.864004,2022-10-20 17:06:19.864004,[order2],"[item4, item5]",[],0
7,7,Item out of stock,2022-10-20 17:11:46.864004,2022-10-20 17:11:46.864004,[],[item5],[],0
8,8,Pick Item,2022-10-20 17:11:47.864004,2022-10-20 17:11:47.864004,[],[item4],[],0
9,9,Pay Order,2022-10-20 17:11:47.864004,2022-10-20 17:11:47.864004,[order2],[],[],0
...,...,...,...,...,...,...,...,...
1220,1220,Pick Item,2022-10-21 23:47:47.867991,2022-10-21 23:47:47.867991,[],[item373],[],47
1240,1240,Load Cargo,2022-10-22 00:18:31.867991,2022-10-22 00:18:31.867991,[],"[item373, item372]",[delivery139],47
1838,1838,Fuel Car,2022-10-29 13:17:48.870968,2022-10-29 13:17:48.870968,[],[],[delivery139],47


In [24]:
# Group the dataframe by event execution number and concatenate the activity names within each group
activity_strings = ocel.log.log.sort_values(['event_execution','event_timestamp']).groupby('event_execution')['event_activity'].apply(lambda x: ','.join(x)).tolist()

In [25]:
activity_strings

['Place Order,Confirm Order,Item out of stock,Pick Item,Pay Order,Reorder Item,Place Order,Confirm Order,Pay Order,Pick Item,Pick Item,Place Order,Confirm Order,Item out of stock,Pick Item,Payment Reminder,Reorder Item,Place Order,Confirm Order,Item out of stock,Reorder Item,Pay Order,Pick Item,Place Order,Confirm Order,Item out of stock,Payment Reminder,Reorder Item,Pick Item,Load Cargo,Place Order,Confirm Order,Item out of stock,Pick Item,Pay Order,Reorder Item,Load Cargo,Place Order,Confirm Order,Pay Order,Pick Item,Pick Item,Load Cargo,Place Order,Confirm Order,Load Cargo,Payment Reminder,Pick Item,Pick Item,Place Order,Confirm Order,Item out of stock,Payment Reminder,Pick Item,Reorder Item,Place Order,Confirm Order,Item out of stock,Item out of stock,Reorder Item,Pay Order,Reorder Item,Place Order,Confirm Order,Item out of stock,Payment Reminder,Pick Item,Reorder Item,Load Cargo,Place Order,Confirm Order,Item out of stock,Payment Reminder,Pick Item,Reorder Item,Load Cargo,Place Or

In [13]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)

In [14]:
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [15]:
log = sample_traces(ocel, ocpn, 1000)

Generate the traces: 100%|██████████| 1000/1000 [00:00<00:00, 27505.08it/s]


In [16]:
log

[['Place Order',
  'Fuel Car',
  'Confirm Order',
  'Pick Item',
  'Item out of stock',
  'Reorder Item',
  'Pick Item',
  'Pay Order',
  'Load Cargo',
  'Start Route',
  'End Route'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Item out of stock',
  'Pay Order',
  'Pick Item',
  'Reorder Item',
  'Pick Item',
  'Load Cargo',
  'Start Route',
  'End Route'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Payment Reminder',
  'Pick Item',
  'Load Cargo',
  'Item out of stock',
  'Payment Reminder',
  'Pay Order',
  'Reorder Item',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Pick Item',
  'Fuel Car',
  'Payment Reminder',
  'Load Cargo',
  'Item out of stock',
  'Payment Reminder',
  'Reorder Item',
  'Pick Item',
  'Payment Reminder',
  'Pay Order',
  'Start Route',
  'Load Cargo',
  'End Route'],
 ['Place Order',
  'Fuel Car',
  'Confirm Order',
  'Pay Order',
  'Item out of stock',
  'Reorder Item',
  'Pick Item',
  'Load Cargo',
  'Start Rou

In [17]:
happy_path__ocel = get_happy_path_log(filename)

happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [18]:
log_happy = sample_traces(ocel, happy_path_ocpn, 5)

Generate the traces: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [19]:
log_happy

[['Place Order',
  'Confirm Order',
  'Pick Item',
  'Pay Order',
  'Load Cargo',
  'Fuel Car',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Pay Order',
  'Pick Item',
  'Load Cargo',
  'Fuel Car',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Pick Item',
  'Load Cargo',
  'Pay Order',
  'Fuel Car',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Pay Order',
  'Pick Item',
  'Load Cargo',
  'Fuel Car',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Pay Order',
  'Pick Item',
  'Load Cargo',
  'Fuel Car',
  'Start Route',
  'End Route']]

In [20]:
ots = ["order","item","delivery"]
flower_ocpn = create_flower_model(filename,ots)

In [21]:
log_flower = sample_traces(ocel, flower_ocpn, 5)

Generate the traces: 100%|██████████| 5/5 [00:00<00:00, 578.86it/s]


In [22]:
log_flower

[['Place Order',
  'Pick Item',
  'Payment Reminder',
  'Pay Order',
  'Pay Order',
  'Reorder Item',
  'End Route',
  'Fuel Car',
  'End Route',
  'Pay Order',
  'Load Cargo',
  'Start Route',
  'Fuel Car',
  'End Route',
  'Fuel Car',
  'Pick Item',
  'Fuel Car',
  'End Route',
  'End Route',
  'Load Cargo',
  'Start Route',
  'Load Cargo',
  'Start Route',
  'Pay Order'],
 ['Start Route',
  'End Route',
  'Pick Item',
  'Pick Item',
  'Pay Order',
  'Pick Item',
  'Payment Reminder',
  'Confirm Order',
  'Confirm Order',
  'End Route',
  'Load Cargo',
  'Place Order',
  'Fuel Car',
  'Load Cargo',
  'Payment Reminder',
  'Start Route',
  'End Route',
  'Payment Reminder',
  'Start Route',
  'Load Cargo',
  'Fuel Car',
  'Place Order',
  'Load Cargo',
  'Confirm Order'],
 ['Fuel Car',
  'Payment Reminder',
  'End Route',
  'Item out of stock',
  'Confirm Order',
  'Fuel Car',
  'Pay Order',
  'Reorder Item',
  'Place Order',
  'Pick Item',
  'Load Cargo',
  'Start Route',
  'Load Car

In [25]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_process_variants/order_variant',object_types = ots )

Processing Variant Nets: 100%|██████████| 12/12 [00:00<00:00, 19988.74it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [26]:
log_variant = sample_traces(ocel, variant_ocpn, 20)

Generate the traces: 100%|██████████| 20/20 [00:00<00:00, 948.43it/s]


In [27]:
log_variant

[['Place Order_6',
  'Confirm Order_6',
  'Pick Item_6',
  'Load Cargo_6',
  'Fuel Car_6',
  'Payment Reminder_6',
  'Start Route_6',
  'End Route_6',
  'Pay Order_6'],
 ['Place Order_10',
  'Confirm Order_10',
  'Pick Item_10',
  'Load Cargo_10',
  'Item out of stock_10',
  'Pay Order_10',
  'Reorder Item_10',
  'Pick Item_10',
  'Load Cargo_10',
  'Fuel Car_10',
  'Start Route_10',
  'End Route_10'],
 ['Fuel Car_8',
  'Place Order_8',
  'Confirm Order_8',
  'Item out of stock_8',
  'Payment Reminder_8',
  'Pick Item_8',
  'Payment Reminder_8',
  'Payment Reminder_8',
  'Reorder Item_8',
  'Payment Reminder_8',
  'Pay Order_8',
  'Load Cargo_8',
  'Pick Item_8',
  'Start Route_8',
  'End Route_8'],
 ['Place Order_6',
  'Confirm Order_6',
  'Payment Reminder_6',
  'Pay Order_6',
  'Pick Item_6',
  'Load Cargo_6',
  'Fuel Car_6',
  'Start Route_6',
  'End Route_6'],
 ['Place Order_11',
  'Confirm Order_11',
  'Pick Item_11',
  'Load Cargo_11',
  'Fuel Car_11',
  'Payment Reminder_11',
 